Para inicair o projeto VAmos importar todas as bibliotecas necessarias e importa o arquivo CSV.

In [145]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from functions import clean_text
import statsmodels.formula.api as sm

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [146]:
df = pd.read_csv("desafio_indicium_imdb.csv")

In [147]:
print(df.head())
df.info()

   Unnamed: 0                                   Series_Title Released_Year  \
0           1                                  The Godfather          1972   
1           2                                The Dark Knight          2008   
2           3                         The Godfather: Part II          1974   
3           4                                   12 Angry Men          1957   
4           5  The Lord of the Rings: The Return of the King          2003   

  Certificate  Runtime                     Genre  IMDB_Rating  \
0           A  175 min              Crime, Drama          9.2   
1          UA  152 min      Action, Crime, Drama          9.0   
2           A  202 min              Crime, Drama          9.0   
3           U   96 min              Crime, Drama          9.0   
4           U  201 min  Action, Adventure, Drama          8.9   

                                            Overview  Meta_score  \
0  An organized crime dynasty's aging patriarch t...       100.0   
1  W

Qual filme você recomendaria para uma pessoa que você não conhece?

Para essa pergunta irei fazer uma analise simples onde pegarei os 10 filmes com maiores notas, depois os 5 com mais votos e por final o com maior media podendera de todas as criticas.

In [148]:
maiores_notas = df.nlargest(10, 'IMDB_Rating')
maiores_votos = maiores_notas.nlargest(5, 'No_of_Votes')
melhor_filme = maiores_votos.nlargest(1, 'Meta_score')
print(melhor_filme['Series_Title'])

5    Pulp Fiction
Name: Series_Title, dtype: object


Eu indicaria o filme Pulp fiction!

----------------------------------------------------------------------------------

Quais são os principais fatores que estão relacionados com alta expectativa de faturamento de um filme? 

Para iniciar irei fazer uma cópia do dataframe, e fazer o tratamentos dos dados numericos.
Nesse teste decidi usar somente os dados numricos para analisar o possivel faturamento.

In [149]:
df_clean = df.copy()

In [150]:
df_clean = df.copy()
df_clean = df.copy()
df_clean['Gross'] = df_clean['Gross'].str.replace(',', '')
df_clean.dropna(subset=['Gross'], inplace=True)
df_clean['Gross'] = df_clean['Gross'].astype(float)
df_clean['Runtime'] = df_clean['Runtime'].str.replace(' min', '')
df_clean['Runtime'] = df_clean['Runtime'].astype(int)

In [151]:
df_clean['Meta_score'].fillna(df_clean['Meta_score'].mean(), inplace=True)

C:\Users\pedro\AppData\Local\Temp\ipykernel_18604\288938973.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['Meta_score'].fillna(df_clean['Meta_score'].mean(), inplace=True)


In [152]:
df_clean['Released_Year'] = pd.to_numeric(df_clean['Released_Year'], errors='coerce')
df_clean.dropna(subset=['Released_Year'], inplace=True)
df_clean['Released_Year'] = df_clean['Released_Year'].astype(int)

In [153]:
features = ['Runtime', 'IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Released_Year']
X = df_clean[features]
y = df_clean['Gross']

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [155]:
model_linear = LinearRegression()
model_linear.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [156]:
coefficients = model_linear.coef_
feature_importance = pd.DataFrame({'feature': X_train.columns, 'coefficient': coefficients})
feature_importance['abs_coefficient'] = feature_importance['coefficient'].abs()
feature_importance = feature_importance.sort_values(by='abs_coefficient', ascending=False)

In [157]:
print('Os 5 principais fatores relacionados com a expectativa de faturamento:')
print(feature_importance[['feature', 'coefficient']].head().to_markdown(index=False, numalign="left", stralign="left"))

Os 5 principais fatores relacionados com a expectativa de faturamento:
| feature       | coefficient   |
|:--------------|:--------------|
| IMDB_Rating   | -1.20879e+08  |
| Meta_score    | 545336        |
| Runtime       | 388603        |
| Released_Year | 152897        |
| No_of_Votes   | 251.367       |


Como podemos ver a coluna que mais afeta o faturamento é a Nota do IMDB!

----------------------------------------------------

Quais insights podem ser tirados com a coluna Overview? É possível inferir o gênero do filme a partir dessa coluna?

esse é um problema de classificação onde farei o tratamento dos dados do Overview para descobrir qual genero é o filme.

In [158]:
df['Overview'].fillna('')

0      An organized crime dynasty's aging patriarch t...
1      When the menace known as the Joker wreaks havo...
2      The early life and career of Vito Corleone in ...
3      A jury holdout attempts to prevent a miscarria...
4      Gandalf and Aragorn lead the World of Men agai...
                             ...                        
994    A young New York socialite becomes interested ...
995    Sprawling epic covering the life of a Texas ca...
996    In Hawaii in 1941, a private is cruelly punish...
997    Several survivors of a torpedoed merchant ship...
998    A man in London tries to help a counter-espion...
Name: Overview, Length: 999, dtype: object

In [159]:
df['clean_overview'] = df['Overview'].apply(clean_text)

Como há alguns filmes com diversos generos tive que fazer a separação de cada um.

In [160]:
df['first_genre'] = df['Genre'].apply(lambda x: x.split(',')[0].strip())

In [161]:
X = df['clean_overview']
y = df['first_genre']

In [162]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_vectorized = vectorizer.fit_transform(X)

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

print("Dimensões dos dados após a vetorização:")
print(f"Características de treinamento (X_train): {X_train.shape}")
print(f"Características de teste (X_test): {X_test.shape}")
print(f"\nNúmero de gêneros únicos: {len(y.unique())}")
print(f"Exemplo de sinopse vetorizada (primeira linha): \n{X_train[0]}")

Dimensões dos dados após a vetorização:
Características de treinamento (X_train): (799, 5000)
Características de teste (X_test): (200, 5000)

Número de gêneros únicos: 14
Exemplo de sinopse vetorizada (primeira linha): 
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 19 stored elements and shape (1, 5000)>
  Coords	Values
  (0, 2639)	0.12356173291859589
  (0, 491)	0.19683670129322098
  (0, 4538)	0.138113876377936
  (0, 4233)	0.21472351176590013
  (0, 4416)	0.2103842142979685
  (0, 3063)	0.21964300149271773
  (0, 2612)	0.21472351176590013
  (0, 302)	0.24026005689522148
  (0, 3982)	0.225322135596595
  (0, 565)	0.225322135596595
  (0, 3767)	0.24026005689522148
  (0, 55)	0.21472351176590013
  (0, 316)	0.2508586807259164
  (0, 4926)	0.2508586807259164
  (0, 3964)	0.26579660202454286
  (0, 2156)	0.26579660202454286
  (0, 239)	0.26579660202454286
  (0, 2921)	0.26579660202454286
  (0, 4495)	0.26579660202454286


In [164]:
model_multi = MultinomialNB()
model_multi.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [165]:
predictions = model_multi.predict(X_test)

In [166]:
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

c:\Users\pedro\OneDrive\Desktop\ML filme\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\pedro\OneDrive\Desktop\ML filme\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\pedro\OneDrive\Desktop\ML filme\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

In [167]:
print(f"Acurácia do modelo: {accuracy:.2f}")
print("\nRelatório de Classificação:")
print(report)

Acurácia do modelo: 0.32

Relatório de Classificação:
              precision    recall  f1-score   support

      Action       0.33      0.06      0.11        31
   Adventure       0.00      0.00      0.00        15
   Animation       0.00      0.00      0.00        18
   Biography       0.00      0.00      0.00        18
      Comedy       0.00      0.00      0.00        28
       Crime       0.00      0.00      0.00        23
       Drama       0.31      0.98      0.48        62
      Horror       0.00      0.00      0.00         3
     Mystery       0.00      0.00      0.00         2

    accuracy                           0.32       200
   macro avg       0.07      0.12      0.06       200
weighted avg       0.15      0.32      0.16       200



Os filmes com a maior dificuldade de se prever pela sinopse é os de Drama!

-----------------------------------------------------

Explique como você faria a previsão da nota do imdb a partir dos dados. 
Quais variáveis e/ou suas transformações você utilizou e por quê?
Qual tipo de problema estamos resolvendo (regressão, classificação)? 
Qual modelo melhor se aproxima dos dados e quais seus prós e contras? 
Qual medida de performance do modelo foi escolhida e por quê?

O modelo que eu escolhi para esse teste foi o de floresta aleatoria um modelo de regressão,com TF-IDF para analisar os textos, escolhi essa abordagem por ter muitos dados aleatorios,
como temos que descobrir o IMDB sobram poucos dados numericos para se trabalhar.

In [168]:
df['Overview'] = df['Overview'].fillna('')

In [169]:
df['Overview_clean'] = df['Overview'].apply(clean_text)

In [170]:
df['features_text'] = df['Series_Title'].fillna('') + ' ' + df['Genre'].fillna('') + ' ' + df['Overview_clean'] + ' ' + df['Director'].fillna('') + ' ' + df['Star1'].fillna('') + ' ' + df['Star2'] + ' ' + df['Star3'].fillna('') + ' ' + df['Star4'].fillna('')

In [171]:
vectorizer = TfidfVectorizer(max_features=30)
X = vectorizer.fit_transform(df['features_text'])
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7708 stored elements and shape (999, 30)>
  Coords	Values
  (0, 22)	0.1741989070070357
  (0, 7)	0.6843174754564637
  (0, 8)	0.1784165289741397
  (0, 2)	0.3160115227696894
  (0, 19)	0.20510719337443306
  (0, 14)	0.5335558124032471
  (0, 25)	0.20716618335245326
  (1, 22)	0.6111288031410864
  (1, 7)	0.24007390570329326
  (1, 8)	0.12518503323117766
  (1, 19)	0.4317371988934995
  (1, 14)	0.18718333578855165
  (1, 25)	0.14535707928222324
  (1, 0)	0.25077999322936045
  (1, 4)	0.29684430400026257
  (1, 3)	0.3037710152138422
  (1, 20)	0.2595525902746082
  (2, 22)	0.3911507079083058
  (2, 7)	0.5121946889291896
  (2, 8)	0.1335403549306978
  (2, 19)	0.15351765646125237
  (2, 14)	0.39935331649677946
  (2, 3)	0.32404583952462374
  (2, 20)	0.276876109977505
  (2, 15)	0.17736747941624423
  :	:
  (996, 20)	0.2729050061345951
  (996, 15)	0.5244707432009985
  (996, 16)	0.25059024472804353
  (996, 11)	0.2929966650276454
  (996, 26)	0.3379658167

In [172]:
y = df['IMDB_Rating']

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [174]:
model_floresta = RandomForestRegressor(n_estimators=100, random_state=42)
model_floresta.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [175]:
y_pred = model_floresta.predict(X_test)
print(y_pred)

[8.127  7.957  7.989  7.974  7.886  8.106  7.912  7.998  8.034  7.933
 7.931  8.017  7.962  8.037  7.964  8.033  7.917  7.878  8.046  7.915
 7.929  7.834  8.119  8.133  7.758  7.88   8.009  7.873  7.886  7.938
 7.98   8.021  7.9    7.991  8.029  8.01   7.948  7.875  7.998  7.9
 7.992  7.891  7.828  7.94   7.877  7.905  8.022  7.966  7.914  7.83
 7.91   8.016  7.873  7.919  7.969  7.944  8.113  7.898  7.944  7.904
 7.897  8.024  7.969  7.872  7.8715 8.054  7.999  8.008  7.853  7.965
 7.935  7.899  7.99   8.029  7.942  7.881  8.156  8.061  8.162  7.878
 8.078  7.959  7.902  7.883  7.87   8.068  8.202  7.88   8.066  8.013
 8.006  8.139  7.806  7.935  8.033  7.947  7.913  7.86   7.957  8.039
 7.907  7.94   7.989  8.068  7.9175 8.083  7.926  7.957  7.954  7.907
 8.157  7.929  7.912  7.957  8.107  8.142  7.913  7.872  7.85   7.991
 7.993  7.839  7.91   7.999  7.773  8.013  7.864  7.923  7.973  8.037
 7.964  7.892  8.044  7.938  8.033  7.95   7.9    8.051  7.969  7.856
 8.02   7.956  7.914  8

Para analisar a performance eu utilizei o erro médio absoluto, como veremos logo abaixo o modelo apresenta um erro de 0,22 do valor real!

In [176]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Erro Médio Absoluto (MAE): {mae:.2f}")

Erro Médio Absoluto (MAE): 0.22


-------------------------------------------------

Supondo um filme com as seguintes características:

{'Series_Title': 'The Shawshank Redemption',
 'Released_Year': '1994',
 'Certificate': 'A',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
 'Meta_score': 80.0,
 'Director': 'Frank Darabont',
 'Star1': 'Tim Robbins',
 'Star2': 'Morgan Freeman',
 'Star3': 'Bob Gunton',
 'Star4': 'William Sadler',
 'No_of_Votes': 2343110,
 'Gross': '28,341,469'}


Qual seria a nota do IMDB?


In [177]:
pergunta4 = [{'Series_Title': 'The Shawshank Redemption',
'Released_Year': '1994',
'Certificate': 'A',
'Runtime': '142 min',
'Genre': 'Drama',
'Overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
'Meta_score': 80.0,
'Director': 'Frank Darabont',
'Star1': 'Tim Robbins',
'Star2': 'Morgan Freeman',
'Star3': 'Bob Gunton',
'Star4': 'William Sadler',
'No_of_Votes': 2343110,
'Gross': '28,341,469'}
]

In [178]:
pergunta4 = pd.DataFrame(pergunta4)
pergunta4['Overview']= pergunta4['Overview'].replace(' ', '')
pergunta4['Overview_clean'] = pergunta4['Overview'].apply(clean_text)

In [179]:
pergunta4['features_text'] = pergunta4['Series_Title'].fillna('') + ' ' + pergunta4['Genre'].fillna('') + ' ' + pergunta4['Overview_clean'] + ' ' + pergunta4['Director'].fillna('') + ' ' + pergunta4['Star1'].fillna('') + ' ' + pergunta4['Star2'] + ' ' + pergunta4['Star3'].fillna('') + ' ' + pergunta4['Star4'].fillna('')

In [180]:
vectorizer1 = TfidfVectorizer(max_features=5000)
X1 = vectorizer1.fit_transform(pergunta4['features_text'])

In [181]:
y_pred1 = model_floresta.predict(X1)
print(y_pred1)

[7.935]


Como podemos ver a nota do IMDB seria 7,94!

In [182]:
import pickle

with open ("modelo_floresta.pkl", "wb") as arquivo:
    pickle.dump(model_floresta, arquivo)